In [1]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer

from sklearn.linear_model import LinearRegression, Lasso, Ridge

# Clean

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/ryskgit/QTW_CaseStudy/main/Case%20Study%201/Raw%20Data%201/train.csv')
material = pd.read_csv('https://raw.githubusercontent.com/ryskgit/QTW_CaseStudy/main/Case%20Study%201/Raw%20Data%201/unique_m.csv')
train.drop(columns="critical_temp", inplace=True)
df = pd.concat([train, material], axis=1)
df.drop(columns='material', inplace=True)

# same way with pd.merge()
# df = pd.merge(train, material, left_index=True, right_index=True)
# df.drop(columns=["critical_temp_y", "material"], inplace=True)
# df.rename(columns={"critical_temp_x":"critical_temp"}, inplace=True)

In [3]:
single_vals = df.columns[df.eq(df.iloc[0]).all()].tolist()
print(f"There are {len(single_vals)} columns with single values: {single_vals}")
print(f"Original data dimiension: {df.shape}")
df.drop(columns=single_vals, inplace=True)
print(f"Final data dim after dropping aforementioned columns: {df.shape}")
print(f"There are {df.isna().sum().sum()} NA values and {df.duplicated().sum()} duplicated records")

There are 9 columns with single values: ['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Pm', 'Po', 'At', 'Rn']
Original data dimiension: (21263, 168)
Final data dim after dropping aforementioned columns: (21263, 159)
There are 0 NA values and 0 duplicated records


In [4]:
df.dtypes.value_counts()

float64    156
int64        3
dtype: int64

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
df.head(5)

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,H,Li,Be,B,C,N,O,F,Na,Mg,Al,Si,P,S,Cl,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Br,Rb,Sr,Y,Zr,Nb,Mo,Tc,Ru,Rh,Pd,Ag,Cd,In,Sn,Sb,Te,I,Cs,Ba,La,Ce,Pr,Nd,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,775.425,1010.268571,718.152900,938.016780,1.305967,0.791488,810.6,735.985714,323.811808,355.562967,160.25,105.514286,136.126003,84.528423,1.259244,1.207040,205,42.914286,75.237540,69.235569,4654.35725,2961.502286,724.953211,53.543811,1.033129,0.814598,8958.571,1579.583429,3306.162897,3572.596624,81.8375,111.727143,60.123179,99.414682,1.159687,0.787382,127.05,80.987143,51.433712,42.558396,6.9055,3.846857,3.479475,1.040986,1.088575,0.994998,12.878,1.744571,4.599064,4.666920,107.756645,61.015189,7.062488,0.621979,0.308148,0.262848,399.97342,57.127669,168.854244,138.517163,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,1.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,766.440,1010.612857,720.605511,938.745413,1.544145,0.807078,810.6,743.164286,290.183029,354.963511,161.20,104.971429,141.465215,84.370167,1.508328,1.204115,205,50.571429,67.321319,68.008817,5821.48580,3021.016571,1237.095080,54.095718,1.314442,0.914802,10488.571,1667.383429,3767.403176,3632.649185,90.8900,112.316429,69.833315,101.166398,1.427997,0.838666,127.05,81.207857,49.438167,41.667621,7.7844,3.796857,4.403790,1.035251,1.374977,1.073094,12.878,1.595714,4.473363,4.603000,172.205316,61.372331,16.064228,0.619735,0.847404,0.567706,429.97342,51.413383,198.554600,139.630922,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0
2,4,88.944468,57.885242,66.361592,36

In [7]:
pd.reset_option("max_columns")

In [8]:
df.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0


# EDA

In [ ]:
# correlation plot

In [8]:
# run this chunk to view histograms of each feature
# commenting out bc output is too long
# for i in df.columns:
#     sns.displot(df[i], kde=True)
#     plt.show()

# Preprocess

After prelim eda, we see that normalcy is met thus standard scaler is the most useful scaler for this data. For completeness, we showed that the internal cross validation metrics between scaled and unscaled data are equivalent.

In [9]:
X = df.drop(columns="critical_temp")
y = df["critical_temp"]

## No scaling

In [10]:
lr = LinearRegression()
lr.fit(X, y)

LinearRegression()

In [11]:
split = KFold(shuffle=True, 
              n_splits=5, 
              random_state=77)
scoring = ["r2", "neg_root_mean_squared_error"]
scores = cross_validate(lr, X, y, 
                        cv=split,
                        scoring=scoring,
                        return_train_score=True,
                        return_estimator=True,
                        n_jobs=-1)
scores = pd.DataFrame(scores)
scores

,fit_time,score_time,estimator,test_r2,train_r2,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error
0,0.241679,0.003327,LinearRegression(),0.770972,0.762466,-16.308913,-16.715386
1,0.232382,0.004211,LinearRegression(),-0.364983,0.769668,-40.298407,-16.409987
2,0.240476,0.003427,LinearRegression(),0.750902,0.767497,-17.031445,-16.532009
3,0.243465,0.009685,LinearRegression(),0.317922,0.764245,-28.201902,-16.644465
4,0.240603,0.003338,LinearRegression(),0.756333,0.765512,-16.989078,-16.567076


In [12]:
# this shows the slopes of each feature from the first fit
# commenting out bc out is too long
# scores["estimator"][0].coef_

In [13]:
# 5 fold internal cv on full data
print("Nonscaled r2: %.5f" % scores["test_r2"].mean())
print("Nonscaled rmse: %.5f" % np.abs(scores["test_neg_root_mean_squared_error"].mean()))

Nonscaled r2: 0.44623
Nonscaled rmse: 23.76595


## Standard Scaler

In [14]:
pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("lr", lr)
])
pipe.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()), ('lr', LinearRegression())])

In [15]:
split = KFold(shuffle=True, 
              n_splits=5, 
              random_state=77)
scoring = ["r2", "neg_root_mean_squared_error"]
scale_scores = cross_validate(pipe, X, y, 
                        cv=split,
                        scoring=scoring,
                        return_train_score=True,
                        return_estimator=True,
                        n_jobs=-1)
scale_scores = pd.DataFrame(scale_scores)
scale_scores

,fit_time,score_time,estimator,test_r2,train_r2,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error
0,0.227199,0.007714,"(StandardScaler(), LinearRegression())",0.770972,0.762466,-16.308913,-16.715386
1,0.219436,0.005143,"(StandardScaler(), LinearRegression())",-0.364983,0.769668,-40.298407,-16.409987
2,0.210362,0.008529,"(StandardScaler(), LinearRegression())",0.750902,0.767497,-17.031445,-16.532009
3,0.232405,0.007150,"(StandardScaler(), LinearRegression())",0.317922,0.764245,-28.201902,-16.644465
4,0.232450,0.007075,"(StandardScaler(), LinearRegression())",0.756333,0.765512,-16.989078,-16.567076


In [16]:
# 5 fold internal cv on full data
print("std scale r2: %.5f" % scores["test_r2"].mean())
print("std scale rmse: %.5f" % np.abs(scores["test_neg_root_mean_squared_error"].mean()))

std scale r2: 0.44623
std scale rmse: 23.76595


# Modeling

## L1 / Lasso Gridsearch

In [20]:
lasso_pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ('model', Lasso(random_state=77))
])

params = {'model__alpha': (np.logspace(-1, 1, 200))} 
split = KFold(shuffle=True, 
              n_splits=5, 
              random_state=77)
scoring = ["neg_root_mean_squared_error", "r2"]
# scoring = {"R2": "r2", "RMSE": make_scorer("neg_root_mean_squared_error", greater_is_better=False)}


lasso_gs = GridSearchCV(lasso_pipe,
						param_grid=params,
						cv=split,
						scoring=scoring,
                        refit="neg_root_mean_squared_error",
						n_jobs=-1
)

In [21]:
lasso_gs.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=77, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso(random_state=77))]),
             n_jobs=-1,
             param_grid={'model__alpha': array([ 0.1       ,  0.10234114,  0.10473709,  0.10718913,  0.10969858,
        0.11226678,  0.1148951 ,  0.11758496,  0.12033778,  0.12315506,
        0.12603829,  0.12898903,  0.13200884,  0.13509935,  0.138262...
        5.11143348,  5.23109931,  5.35356668,  5.47890118,  5.60716994,
        5.73844165,  5.87278661,  6.01027678,  6.15098579,  6.29498899,
        6.44236351,  6.59318827,  6.74754405,  6.90551352,  7.06718127,
        7.2326339 ,  7.40196   ,  7.57525026,  7.75259749,  7.93409667,
        8.11984499,  8.30994195,  8.50448934,  8.70359136,  8.90735464,
        9.1158883 ,  9.32930403,  9.54771611,  9.77124154, 10.        ])},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2'])

In [22]:
lasso_results = pd.DataFrame(lasso_gs.cv_results_).sort_values("mean_test_neg_root_mean_squared_error", ascending=False)
lasso_results[
    [
        "mean_test_neg_root_mean_squared_error",
        "mean_test_r2",
        "param_model__alpha"
    ]
].head(5)

,mean_test_neg_root_mean_squared_error,mean_test_r2,param_model__alpha
130,-20.087865,0.655845,2.025502
129,-20.089264,0.655723,1.979167
131,-20.090978,0.655796,2.072922
128,-20.091017,0.655580,1.933892
127,-20.096490,0.655293,1.889652


### Feature importance

In [23]:
lasso_feat_imp = pd.DataFrame({"Parameter":lasso_gs.best_estimator_.named_steps["model"].coef_, "Feature":lasso_gs.best_estimator_[:-1].get_feature_names_out()}, 
                             index=lasso_gs.best_estimator_[:-1].get_feature_names_out()
)

In [24]:
lasso_feat_imp["Parameter_Rank"] = ((lasso_feat_imp["Parameter"].abs().to_numpy()*-1).argsort().argsort() + 1)
lasso_feat_imp = lasso_feat_imp.reindex(lasso_feat_imp["Parameter"].abs().sort_values(ascending=False).index)
lasso_feat_imp.reset_index(drop=True, inplace=True)
lasso_feat_imp.head(20)

,Parameter,Feature,Parameter_Rank
0,11.416835,wtd_std_ThermalConductivity,1
1,7.531124,Ba,2
2,3.185943,wtd_entropy_atomic_mass,3
3,3.115142,Ca,4
4,-2.097953,wtd_gmean_ElectronAffinity,5
5,-2.014136,wtd_std_Valence,6
6,1.686567,Bi,7
7,1.538015,range_atomic_radius,8
8,1.368411,range_fie,9
9,-1.269696,Si,10


## L2 / Ridge Gridsearch

In [25]:
ridge_pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ('model', Ridge(random_state=77))
])

params = {'model__alpha': (np.logspace(-1, 2, 300))} 
split = KFold(shuffle=True, 
              n_splits=5, 
              random_state=77)
scoring = ["neg_root_mean_squared_error", "r2"]
# scoring = {"R2": "r2", "RMSE": make_scorer("neg_root_mean_squared_error", greater_is_better=False)}


ridge_gs = GridSearchCV(ridge_pipe,
						param_grid=params,
						cv=split,
						scoring=scoring,
                        refit="neg_root_mean_squared_error",
						n_jobs=-1
)

In [26]:
ridge_gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=77, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Ridge(random_state=77))]),
             n_jobs=-1,
             param_grid={'model__alpha': array([  0.1       ,   0.10233718,   0.10472898,   0.10717669,
         0.1096816 ,   0.11224506,   0.11486843,   0.11755311,
         0.12030053,   0.12311217,   0.12598953,   0.12893413,
         0.13194755,   0.1350314 ,   0.1381873...
        58.78016072,  60.1539588 ,  61.55986501,  62.99862978,
        64.47102107,  65.9778248 ,  67.51984523,  69.09790546,
        70.71284779,  72.36553423,  74.05684692,  75.78768864,
        77.55898323,  79.37167617,  81.226735  ,  83.1251499 ,
        85.06793416,  87.05612479,  89.09078301,  91.17299485,
        93.30387173,  95.48455103,  97.71619673, 100.        ])},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2'])

In [27]:
ridge_results = pd.DataFrame(ridge_gs.cv_results_).sort_values("mean_test_neg_root_mean_squared_error", ascending=False)
ridge_results[
    [
        "mean_test_neg_root_mean_squared_error",
        "mean_test_r2",
        "param_model__alpha"
    ]
].head(5)

,mean_test_neg_root_mean_squared_error,mean_test_r2,param_model__alpha
299,-22.430580,0.533886,100.0
298,-22.434626,0.533604,97.716197
297,-22.438754,0.533319,95.484551
296,-22.442965,0.533029,93.303872
295,-22.447259,0.532734,91.172995


### Feature importance

In [28]:
ridge_feat_imp = pd.DataFrame({"Parameter":ridge_gs.best_estimator_.named_steps["model"].coef_, "Feature":ridge_gs.best_estimator_[:-1].get_feature_names_out()}, 
                             index=ridge_gs.best_estimator_[:-1].get_feature_names_out()
)

In [29]:
ridge_feat_imp["Parameter_Rank"] = ((ridge_feat_imp["Parameter"].abs().to_numpy()*-1).argsort().argsort() + 1)
ridge_feat_imp = ridge_feat_imp.reindex(ridge_feat_imp["Parameter"].abs().sort_values(ascending=False).index)
ridge_feat_imp.reset_index(drop=True, inplace=True)
ridge_feat_imp.head(20)

,Parameter,Feature,Parameter_Rank
0,14.107843,wtd_mean_ThermalConductivity,1
1,12.494664,std_ElectronAffinity,2
2,-9.661939,range_ElectronAffinity,3
3,8.592817,Ba,4
4,7.840255,range_atomic_mass,5
5,-7.663575,wtd_mean_atomic_mass,6
6,7.607960,wtd_std_ThermalConductivity,7
7,-7.507625,wtd_gmean_ElectronAffinity,8
8,-7.451777,wtd_gmean_ThermalConductivity,9
9,-7.024038,std_atomic_radius,10
